<a href="https://colab.research.google.com/github/IgnacioLY9/RosterToolPublic/blob/main/BuildRoster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# necessary imports
from itertools import combinations
import numpy as np
from google.colab import files

--------------------------------------------------------------------------------

Only the next two cells require user input

In [90]:
# change the following two values to suit your needs

# NumberOfAthletesOnTeam is the maximum number of athletes than can compete
# for example, for an NCAA team, NumberOfAthletesOnTeam should be set to 10
# for Competitions like World Championships, it should be set to 5
NumberOfAthletesOnTeam = 7

# NumberOfAthletesPerEvent is the number of athletes that must compete on an
# event. For NCAA, this is 4. For something like the Team Final at World
# Champtionships, this is 3.
NumberOfAthletesPerEvent = 6

In [ ]:
# allows the user to select the csv file containing team data
fileInput = list(files.upload().keys())[0]
data = np.genfromtxt(fileInput, delimiter=',', dtype= 'str')
# there should be a white button below this cell that can be used to
# to select rosterInput.csv

--------------------------------------------------------------------------------

In [ ]:
# class defining a roster. The roster consists of a list of athletes
class Roster:
    def __init__(self):
        self.athletes = []

    def __str__(self):
        str = ""
        for ath in self.athletes:
            str = str + ath.name + " "
        return str

In [ ]:
# class defining an athlete. The athlete has a name and a score for each of the 6 events
class Athlete:
    def __init__(self, name, scores):
        self.name = name
        self.fx = scores[0]
        self.ph = scores[1]
        self.sr = scores[2]
        self.vt = scores[3]
        self.pb = scores[4]
        self.hb = scores[5]

In [ ]:
# helper function
# for each event, we take every athlete score in the squad
# then we select the 4 best scores on each event and rank them
def optimum(squad):
    people = []

    rosterSize = len(squad.athletes)
    floorScores = []
    horseScores = []
    ringsScores = []
    vaultScores = []
    pbarScores = []
    hbarScores = []
    for athlete in squad.athletes:
        floorScores.append(athlete.fx)
        horseScores.append(athlete.ph)
        ringsScores.append(athlete.sr)
        vaultScores.append(athlete.vt)
        pbarScores.append(athlete.pb)
        hbarScores.append(athlete.hb)
    floorScores = np.array(floorScores)
    horseScores = np.array(horseScores)
    ringsScores = np.array(ringsScores)
    vaultScores = np.array(vaultScores)
    pbarScores = np.array(pbarScores)
    hbarScores = np.array(hbarScores)

    fxidx = np.argsort(floorScores)[::-1]
    phidx = np.argsort(horseScores)[::-1]
    sridx = np.argsort(ringsScores)[::-1]
    vtidx = np.argsort(vaultScores)[::-1]
    pbidx = np.argsort(pbarScores)[::-1]
    hbidx = np.argsort(hbarScores)[::-1]

    if (rosterSize < NumberOfAthletesPerEvent):
        people.extend([fxidx, phidx, sridx, vtidx, pbidx, hbidx])
    else:
        people.extend([fxidx[0:NumberOfAthletesPerEvent], phidx[0:NumberOfAthletesPerEvent], sridx[0:NumberOfAthletesPerEvent],
                       vtidx[0:NumberOfAthletesPerEvent], pbidx[0:NumberOfAthletesPerEvent], hbidx[0:NumberOfAthletesPerEvent]])
    return people

In [ ]:
# helper function to calculate the score of a specific squad
# just calculate each event score, and then add up all of the
# event scores to get the team score
# returns a list consisting of the 6 event scores and the total
def teamScore(squad):
    fxscore = 0
    phscore = 0
    srscore = 0
    vtscore = 0
    pbscore = 0
    hbscore = 0

    people = optimum(squad)
    for ath in people[0]:
        fxscore = fxscore + squad.athletes[ath].fx
    for ath in people[1]:
        phscore = phscore + squad.athletes[ath].ph
    for ath in people[2]:
        srscore = srscore + squad.athletes[ath].sr
    for ath in people[3]:
        vtscore = vtscore + squad.athletes[ath].vt
    for ath in people[4]:
        pbscore = pbscore + squad.athletes[ath].pb
    for ath in people[5]:
        hbscore = hbscore + squad.athletes[ath].hb

    total = fxscore + phscore + srscore + vtscore + pbscore + hbscore

    scores = [fxscore, phscore, srscore, vtscore, pbscore, hbscore, total]
    return scores

In [ ]:
# helper function that creates all possible combinations of athletes
# of the specified size
# it calculates all of the expected scores for those teams and returns
# a sorted list of those squads by score
def makeTeams(Team):
    n = len(Team.athletes)

    playerSelection = []
    for ones_positions in combinations(range(n), NumberOfAthletesOnTeam):
        binary_string = ['0'] * n
        for pos in ones_positions:
            binary_string[pos] = '1'
        playerSelection.append(''.join(binary_string))

    scores = []
    for selection in playerSelection:
        squad = Roster()
        for i in range(0,n):
            if selection[i] == "1":
                squad.athletes.append(Team.athletes[i])
        scores.append(teamScore(squad))

    returnList = []

    for x, y in zip(playerSelection, scores):
        returnList.append((x,y))

    returnList = sorted(returnList, key = lambda element: element[1][6], reverse = True)

    return returnList


In [ ]:
# read the data from the numpy array
firstCol = data[:,0]
fxidx = np.where(firstCol == "Floor")[0][0]
phidx = np.where(firstCol == "Horse")[0][0]
sridx = np.where(firstCol == "Rings")[0][0]
vtidx = np.where(firstCol == "Vault")[0][0]
pbidx = np.where(firstCol == "Pbars")[0][0]
hbidx = np.where(firstCol == "Hbar")[0][0]

FloorTable = data[1: phidx]
HorseTable = data[phidx: sridx]
RingsTable = data[sridx: vtidx]
VaultTable = data[vtidx: pbidx]
PbarsTable = data[pbidx: hbidx]
HbarTable = data[hbidx:]

Events = [FloorTable, HorseTable, RingsTable, VaultTable, PbarsTable, HbarTable]

Names = np.unique(firstCol)
exclude = ['', 'Floor', 'Horse', 'Rings', 'Vault', 'Pbars', 'Hbar']
Names = Names[~np.isin(Names, exclude)]

Team = Roster()

In [ ]:
# helper function that returns an individual athlete's expected score
# for a specified event
def eventScore(weight, loc, event):
    # loc of -1 means that there are no scores for this athlete
    # on this event at all
    if loc == -1:
        return 0
    # load the weights
    # the get loaded as string, so convert to float
    # normalize
    weight = weight.astype(float)
    weight = weight / np.sum(weight)

    # Load the numpy array of scores. Convert to float because it
    # is string be default
    scores = event[loc, 1:]
    scores = scores.astype(float)

    # Mask to exclude scores that are -1
    valid_mask = scores != -1

    # Eliminate comps that we don't want to count
    filtered_scores = scores[valid_mask]
    filtered_weights = weight[valid_mask]

    # Re-normalize weights to sum to 1 after the mask has been used
    filtered_weights = filtered_weights / np.sum(filtered_weights)

    # Return the athlete's score
    score = np.dot(filtered_scores, filtered_weights)
    return score

In [ ]:
# helper function that calculates the expected scores for each athlete
# and adds the athlete to the athlete list of the Roster
def getAthleteData():
    for gymnast in Names:
        scores = []
        for event in Events:
            weight = event[0, 1:] # all of the weights for that event
            loc = np.where(event[:,0] == gymnast) # find the location of the athlete in the table
            if (loc[0].size != 0): # if the athlete is there, set the location to int
                loc = loc[0][0]
            else: # mark that the athlete is not there with -1
                loc = -1
            scores.append(eventScore(weight, loc, event))
        ath = Athlete(gymnast, np.array(scores))
        Team.athletes.append(ath)

In [ ]:
# get all of the scores for each athlete
getAthleteData()

# if there aren't enough athletes, we can just stop
if (len(Team.athletes) < NumberOfAthletesOnTeam):
  raise Exception("There are less athletes than positions on the team. You can just select all athletes.")

# create every single team
bestTeams = makeTeams(Team)
n = len(Team.athletes)

output = np.full((10 * (3 + NumberOfAthletesPerEvent), 6), '', dtype='<U10')

for i in range(0,  10 if len(bestTeams) > 10 else len(bestTeams)):
    firstline = np.array([f"Team {i+1}", bestTeams[i][1][6], '', '', '', ''])
    # get the ith roster
    output[(3 + NumberOfAthletesPerEvent)*i, :] = firstline
    squad = Roster()
    for j in range(0,n):
            if bestTeams[i][0][j] == "1":
                squad.athletes.append(Team.athletes[j])
    people = optimum(squad)
    output[(3 + NumberOfAthletesPerEvent)*i + 1, :] = np.array(['Floor', 'Horse', 'Rings', 'Vault', 'Pbars', 'Hbar'])
    for j in range(0,6):
        output[(3 + NumberOfAthletesPerEvent)*i+2: (3 + NumberOfAthletesPerEvent)*i+2+NumberOfAthletesPerEvent, j] = np.array([Team.athletes[indices].name for indices in people[j]])
    output[(3 + NumberOfAthletesPerEvent)*i+2+NumberOfAthletesPerEvent, :] = bestTeams[i][1][0:6]

In [ ]:
# Save the results to a file.
np.savetxt('rosterOutput.csv', output, delimiter=',', fmt='%s')
files.download('rosterOutput.csv')